In [5]:
!pip install Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [Levenshtein]


In [6]:
!pip install rouge


In [7]:
import os
import json
import math
import glob
import argparse
from pathlib import Path
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import tree_sitter_java
from tree_sitter import Language, Parser, Query
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList, StoppingCriteria
import Levenshtein
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from tqdm.auto import tqdm
tqdm.pandas()
import re
import random
from typing import List, Tuple
random.seed(42)
import gc
from collections import Counter

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
from google.colab import drive


In [10]:
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
dataset_path = "/content/drive/MyDrive/Personal_RP/heap_Java_sampled_english"


In [12]:
from datasets import load_from_disk, Dataset


In [13]:
ds = load_from_disk(dataset_path)


In [14]:
JAVA_LANGUAGE = Language(tree_sitter_java.language())
parser = Parser(JAVA_LANGUAGE)
COMMENT_JAVA_QUERY = JAVA_LANGUAGE.query(
    """
    (block_comment)       @comment.block
    (line_comment)        @comment.line
    """
)

In [15]:
import random

In [16]:
def parse_code(input_code):
    encoded_code = bytes(input_code, 'utf8')
    tree = parser.parse(encoded_code)
    root_node = tree.root_node


    captures: dict[str, list] = COMMENT_JAVA_QUERY.captures(root_node)
    # ← this is a bytes object
    results = []

    for cap_name, nodes in captures.items():
        for node in nodes:
            start = node.start_byte
            end = node.end_byte

            c_start = len(encoded_code[:start].decode('utf-8', errors='replace'))
            c_end = len(encoded_code[:end].decode('utf-8', errors='replace'))
            snippet = input_code[c_start:c_end]

            results.append(
                (cap_name, c_start, c_end, snippet)
            )

    return results

In [17]:
!pip install -qU "transformers>=4.41.0" accelerate bitsandbytes sentencepiece safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 148.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 196.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [bitsandbytes]


In [18]:
class Config:
    MODELS = {
        'mellum_base_4b': {
            'path': 'JetBrains/Mellum-4b-base',
            'context_length': 2048,
            'fim_prefix': '<fim_prefix>',
            'fim_suffix': '<fim_suffix>',
            'fim_middle': '<fim_middle>',
        },
        'smol_lm_135m': {
            'path': 'HuggingFaceTB/SmolLM-135M',
            'context_length': 2048,
        },
        'starcoder2_3b': {
            'path': 'bigcode/starcoder2-3b',
            'context_length': 2048,
            'fim_prefix': '<fim_prefix>',
            'fim_suffix': '<fim_suffix>',
            'fim_middle': '<fim_middle>',
        },
    }

In [19]:
class FIMInput:
    def __init__(self, FIM_PREFIX = '<fim_prefix>', FIM_SUFFIX = '<fim_suffix>', FIM_MIDDLE = '<fim_middle>'):
        self.FIM_PREFIX = FIM_PREFIX
        self.FIM_SUFFIX = FIM_SUFFIX
        self.FIM_MIDDLE = FIM_MIDDLE

    def generate(self, query_tuple: Tuple[str, str, str]):
        prefix = query_tuple[0]
        suffix = query_tuple[1]
        middle = query_tuple[2]

        text = self.FIM_PREFIX + prefix + self.FIM_SUFFIX + suffix + self.FIM_MIDDLE
        return text, middle

In [20]:
def find_next_line(code: str, c_end: int) -> Tuple[int,int]:
    # split into lines, tracking cumulative lengths
    lines = code.splitlines(keepends=True)
    cum = 0
    for i, ln in enumerate(lines):
        prev_cum = cum
        cum += len(ln)
        # did our comment end in this line?
        if prev_cum <= c_end < cum:
            # scan forward for the next non-blank line
            for j in range(i+1, len(lines)):
                if lines[j].strip():     # non-empty
                    start = sum(len(x) for x in lines[:j])
                    end   = start + len(lines[j])
                    return start, end
            break
    # fallback: no next line
    return c_end, c_end

In [21]:
from typing import Dict, Tuple

def estimate_target_tokens_next_line(
    ds: Dataset,
    tokenizer_name: str,
    parse_fn,
) -> int:
    """
    Scan the dataset once to estimate mean+2*std of token lengths
    for all parsed snippets (e.g. comments).
    Returns the ceiling of mean + 2*std.
    """
    tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
    lengths = []

    for sample in ds:
        code = sample["content"]
        annotations = sample.get("language_detected", [])
        for _, c_start, c_end, _ in parse_fn(code):
            # slice out the per-char language tags for this comment
            comment_ann = annotations[c_start:c_end]
            # drop any “-1” (undetected) labels
            langs = [l for l in comment_ann if l != "-1"]
            if not langs:
                continue

            # find the single most‐common language in the span
            top_lang, _ = Counter(langs).most_common(1)[0]
            if top_lang != "en":
                continue  # skip non-English comments

            # grab the *next* non-empty line of code
            nl_start, nl_end = find_next_line(code, c_end)
            snippet = code[nl_start:nl_end]
            ids = tok(snippet, return_attention_mask=False)["input_ids"]
            lengths.append(len(ids))

    mean, std = np.mean(lengths), np.std(lengths)
    return int(np.ceil(mean + 2 * std))


def get_context_size(
    *,
    context_length: int,
    target_tokens: int,
    supports_fim: bool
) -> Tuple[int, int]:
    """
    Given a model's context window and a fixed target size,
    return (prefix_tokens, suffix_tokens).
    If supports_fim=False, suffix_tokens will be 0.
    """
    if supports_fim:
        rem = context_length - target_tokens
        pre = rem // 2
        suf = rem - pre
    else:
        pre, suf = context_length - target_tokens, 0

    return pre, suf


In [22]:
target_size = estimate_target_tokens_next_line(ds,Config.MODELS['mellum_base_4b']['path'],parse_fn=parse_code)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.65k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/915k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

In [23]:
prefix, suffix = get_context_size(
    context_length = Config.MODELS['mellum_base_4b']['context_length'],
    target_tokens  = target_size,
    supports_fim   = False,
)

In [24]:
def make_target_context(ds: Dataset, prefix_size: int, parse_fn):

    context_target_pair = []
    for sample in ds:
        code = sample["content"]
        annotations = sample.get("language_detected", [])

        for _, c_start, c_end, _ in parse_fn(code):
            # slice out the per-char language tags for this comment
            comment_ann = annotations[c_start:c_end]
            # drop any “-1” (undetected) labels
            langs = [l for l in comment_ann if l != "-1"]
            if not langs:
                continue

            # find the single most‐common language in the span
            top_lang, _ = Counter(langs).most_common(1)[0]

            if top_lang != "en":
                continue  # skip non-English comments

            new_line_start, new_line_end = find_next_line(code, c_end)

            prefix  = code[max(0, new_line_start - prefix_size) : new_line_start]
            target = code[new_line_start : new_line_end]

            if prefix and target:
                context_target_pair.append((prefix, target))

    return context_target_pair


In [25]:
context_target_pair = make_target_context(ds, prefix, parse_fn=parse_code)

In [26]:
class ModelWrapper:
    def __init__(self, config, device, local=False):
        self.device = device
        repo = config['path_local'] if local else config['path']
        self.tokenizer = AutoTokenizer.from_pretrained(str(repo), local_files_only=local)
        self.model = AutoModelForCausalLM.from_pretrained(str(repo), local_files_only=local).to(device)
        self.context_length = config['context_length']

        # initialize FIM tool if tokens provided
        if all(k in config for k in ('fim_prefix', 'fim_suffix', 'fim_middle')):
            self.fim_tool = FIMInput(
                FIM_PREFIX = config['fim_prefix'], FIM_SUFFIX = config['fim_suffix'], FIM_MIDDLE =config['fim_middle']
            )
        else:
            self.fim_tool = None


    def generate(self, input_ids, max_new_tokens: int, attention_mask=None, stopping_criteria=None, pad_token_id=None, eos_token_id=None) -> torch.LongTensor:
        return self.model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            attention_mask=attention_mask,
            # temperature=0.0,
            do_sample = False,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
        )


In [27]:
def first_line(text: str) -> str:
    """
    Return the first non-empty line from `text`.
    If no such line exists, return an empty string.
    """
    for line in text.splitlines()[:-1]:
        if line.strip():
            return line.strip()
    return ""

In [28]:
class StopOnLineGenerated(StoppingCriteria):
    def __init__(self, tokenizer, offset):
        super().__init__()
        self.tokenizer = tokenizer
        self.offset = offset

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        ids = input_ids[0][self.offset:]
        pred_text = self.tokenizer.decode(ids, skip_special_tokens = True)
        if first_line(pred_text):
            return True
        return False

class StopOnCommentGenerated(StoppingCriteria):
    def __init__(self, tokenizer, offset):
        super().__init__()
        self.tokenizer = tokenizer
        self.offset = offset

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        ids = input_ids[0][self.offset:]
        pred_text = self.tokenizer.decode(ids, skip_special_tokens = True)
        return '*/' in pred_text

In [29]:
class PipelineTool:
    def __init__(self, model, fin_mask_token: str = None):
        self.model = model

    def  task_line_completion(self, context: str, line: str, max_tokens: int = 160) -> Tuple[List[int], List[str]]:


        context_tokens = self.model.tokenizer(context, return_tensors='pt').to(self.model.device)
        context_ids = context_tokens['input_ids'][0][-self.model.context_length:].unsqueeze(0)
        context_attention_mask = context_tokens['attention_mask'][0][-self.model.context_length:].unsqueeze(0)


        line_tokenized = self.model.tokenizer(line, return_tensors='pt').to(self.model.device)
        line_ids = line_tokenized['input_ids'][0].unsqueeze(0)
        line_mask = line_tokenized['attention_mask']

        stopping_criterion = StopOnLineGenerated(self.model.tokenizer, context_ids.shape[-1])
        stopping_criteria = StoppingCriteriaList([stopping_criterion])

        gen = self.model.generate(
            input_ids=context_ids,
            attention_mask=context_attention_mask,
            max_new_tokens=max_tokens,
            pad_token_id=self.model.tokenizer.eos_token_id,
            eos_token_id=self.model.tokenizer.pad_token_id,
            stopping_criteria=stopping_criteria
        )

        truth_ids = line_ids[0]

        context_ids = context_ids.detach().cpu()
        context_attention_mask = context_attention_mask.detach().cpu()

        generated_line_ids = gen[0][context_ids.shape[-1]:]

        truth_toks = self.model.tokenizer.convert_ids_to_tokens(truth_ids, skip_special_tokens = True)
        pred_toks = self.model.tokenizer.convert_ids_to_tokens(generated_line_ids, skip_special_tokens = True)

        truth_text = self.model.tokenizer.decode(truth_ids, skip_special_tokens = True)
        pred_text = self.model.tokenizer.decode(generated_line_ids, skip_special_tokens = True)

        gen = gen.detach().cpu()
        line_ids = line_ids.detach().cpu()
        line_mask = line_mask.detach().cpu()

        generated_line_ids = np.array(generated_line_ids.detach().cpu().tolist())
        truth_ids = np.array(truth_ids.detach().cpu().tolist())

        del context_ids
        del context_attention_mask
        del gen
        del line_ids
        del line_mask

        return truth_ids, generated_line_ids, truth_toks, pred_toks, truth_text, pred_text

    def task_fim(self, context: str, target: str, max_tokens:int = 1500):
        context_tokenized = self.model.tokenizer(context, return_tensors = 'pt').to(self.model.device)
        context_ids = context_tokenized['input_ids']
        context_mask = context_tokenized['attention_mask']

        target_tokenized = self.model.tokenizer(target, return_tensors = 'pt').to(self.model.device)
        target_ids = target_tokenized['input_ids']
        target_mask = target_tokenized['attention_mask']

        stopping_criterion = StopOnCommentGenerated(self.model.tokenizer, context_ids.shape[-1])
        stopping_criteria = StoppingCriteriaList([stopping_criterion])

        gen = self.model.generate(
            input_ids=context_ids,
            attention_mask=context_mask,
            max_new_tokens=max_tokens,
            pad_token_id=self.model.tokenizer.eos_token_id,
            eos_token_id=self.model.tokenizer.pad_token_id,
            stopping_criteria=stopping_criteria
        )

        pred_ids = gen[0][context_ids.shape[-1]:]
        pred_toks = self.model.tokenizer.convert_ids_to_tokens(pred_ids, skip_special_tokens = True)
        pred_text = self.model.tokenizer.decode(pred_ids, skip_special_tokens = True)

        truth_ids = target_ids[0]
        truth_toks = self.model.tokenizer.convert_ids_to_tokens(truth_ids, skip_special_tokens = True)
        truth_text = self.model.tokenizer.decode(truth_ids, skip_special_tokens = True)

        pred_ids = np.array(pred_ids.detach().cpu().tolist())
        truth_ids = np.array(truth_ids.detach().cpu().tolist())

        context_ids = context_ids.detach().cpu()
        context_mask = context_mask.detach().cpu()
        target_ids = target_ids.detach().cpu()
        target_mask = target_mask.detach().cpu()
        gen = gen.detach().cpu()

        del context_ids
        del context_mask
        del target_ids
        del target_mask
        del gen

        gc.collect()


        return truth_ids, pred_ids, truth_toks, pred_toks, truth_text, pred_text


In [30]:
class Evaluator:
    @staticmethod
    def sentence_bleu(pred_text: List[str], ref_text: List[str]) -> float:
        """
        Compute the BLEU score for a single sentence.
        Args:
            pred_text (List[str]): List of predicted tokens.
            ref_text (List[str]): List of reference tokens.
        Returns:
            float: BLEU score.
        """
        try:
            chencherry = SmoothingFunction()
            return sentence_bleu([ref_text], pred_text,smoothing_function = chencherry.method2)
        except Exception as e:
            print(f"Error in sentence_bleu: {e}")
            return np.nan

    @staticmethod
    def lev_total(preds: List[str], refs: List[str]) -> float:
        """
        Compute the total Levenshtein distance between two lists of strings. This is the same as edit distance.
        Args:
            preds (List[str]): List of predicted strings.
            refs (List[str]): List of reference strings.
        Returns:
            float: Total Levenshtein distance (normalized per character).
        """
        try:
            length = max(len(preds), len(refs))
            preds_p = preds + [""] * (length - len(preds))
            refs_p  = refs  + [""] * (length - len(refs))

            total_dist = sum(Levenshtein.distance(p, r) for p, r in zip(preds_p, refs_p))
            total_chars = sum(max(len(p), len(r)) for p, r in zip(preds_p, refs_p))
            return total_dist / total_chars if total_chars > 0 else 0.0

        except Exception as e:
            print(f"Error in lev_total: {e}")
            return np.nan

    @staticmethod
    def meteor_score(pred_tokens: List[str], ref_tokens: List[str]) -> float:
        try:
            return meteor_score([ref_tokens], pred_tokens)
        except Exception as e:
            print(f"Error in meteor_score: {e}")
            return np.nan

    @staticmethod
    def rouge_score(pred_text: str, ref_text: str) -> List[float]:
        try:
            rouge = Rouge()
            rouge_scores = rouge.get_scores(pred_text, ref_text)
            res = [
                rouge_scores[0]['rouge-1']['p'],
                rouge_scores[0]['rouge-1']['r'],
                rouge_scores[0]['rouge-1']['f'],
                rouge_scores[0]['rouge-2']['p'],
                rouge_scores[0]['rouge-2']['r'],
                rouge_scores[0]['rouge-2']['f'],
                rouge_scores[0]['rouge-l']['p'],
                rouge_scores[0]['rouge-l']['r'],
                rouge_scores[0]['rouge-l']['f'],
            ]
            return res
        except Exception as e:
            print(f"Error in rouge_score: {e}")
            return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

    def exact_match(generated_ids: np.ndarray, ground_truth_ids: np.ndarray) -> float:
        """
        Compute exact match accuracy between two 1D numpy arrays.
        Pads the shorter one with -1 so lengths match.
        Returns a float ∈ [0, 1].
        """
        try:
            if generated_ids.ndim != 1 or ground_truth_ids.ndim != 1:
                raise ValueError("Both inputs must be 1D arrays.")

            len_gt = ground_truth_ids.shape[0]
            len_gen = generated_ids.shape[0]

            if len_gt != len_gen:
                target_len = max(len_gt, len_gen)
                pad_gt = np.full((target_len - len_gt,), -1, dtype=ground_truth_ids.dtype)
                pad_gen = np.full((target_len - len_gen,), -1, dtype=generated_ids.dtype)
                ground_truth_ids = np.concatenate([ground_truth_ids, pad_gt])
                generated_ids = np.concatenate([generated_ids, pad_gen])

            matches = np.sum(ground_truth_ids == generated_ids)
            total = ground_truth_ids.shape[0]

            return matches / total if total > 0 else 0.0
        except Exception as e:
            print(f"Error in exact_match: {e}")
            return np.nan

    @staticmethod
    def get_results(pred_ids: torch.Tensor, pred_tokens: List[str], pred_text: str,
                    ref_ids: torch.Tensor, ref_tokens: List[str], ref_text: str, device: str) -> List[float]:
        """
        Compute evaluation metrics and return as a tuple of 13 floats.
        """
        exact_match = Evaluator.exact_match(pred_ids, ref_ids)
        levenshtein_distance = Evaluator.lev_total(pred_tokens, ref_tokens)
        meteor = Evaluator.meteor_score(pred_tokens, ref_tokens)
        rouge_scores = Evaluator.rouge_score(pred_text, ref_text)
        sentence_bleu = Evaluator.sentence_bleu(pred_tokens, ref_tokens)

        return np.array([
            exact_match,
            sentence_bleu,
            levenshtein_distance,
            meteor,
            *rouge_scores
        ])

    @staticmethod
    def get_default_results() -> List[float]:
        """
        Return a list of default results (all NaN).
        """
        return np.array([np.nan] * 13)

In [31]:
model = ModelWrapper(Config.MODELS["mellum_base_4b"], "cuda")
tool = PipelineTool(model)
evaluator = Evaluator()

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [32]:
def first_non_empty_line(text: str) -> str:
    """
    Return the first non-empty line from `text`.
    If no such line exists, return an empty string.
    """
    for line in text.splitlines():
        if line.strip():
            return line.strip()
    return ""


In [33]:
results = []
evaluation_results = []
ctx_len = model.context_length
tok     = model.tokenizer

for i, (prefix, target) in enumerate(context_target_pair):
    tok_out = tok(prefix, return_tensors="pt", add_special_tokens=False)
    ids     = tok_out["input_ids"][0]
    used    = ids.size(0)

    if used >= ctx_len:
        ids    = ids[-(ctx_len - 1):]
        prefix = tok.decode(ids, skip_special_tokens=True)
        used   = ids.size(0)

    max_new = ctx_len - used

    try:
        truth_ids, pred_ids, truth_tokens, pred_toks, truth_text, pred_text = \
            tool.task_line_completion(
                context   = prefix,
                line      = target,
                max_tokens= max_new,
            )

        first_generated_line = first_non_empty_line(pred_text)

        toks = tool.model.tokenizer.convert_ids_to_tokens(ids, skip_special_tokens = True)
        ids = np.array(ids.detach().cpu().tolist())

        eval_res = evaluator.get_results(
                  ids, toks, first_generated_line,
                  truth_ids, truth_tokens, truth_text,
                  tool.model.device
              )

        evaluation_results.append(eval_res)
        results.append({
            "prefix":     prefix,
            "truth": truth_text,
            "prediction": first_generated_line,
        })
    except Exception as e:
        print(f"[{i}] generation/eval failed:", str(e))
        evaluation_results.append(Evaluator.get_default_results())
        results.append({
            "prefix":      None,
            "truth_line":  None,
            "prediction":  None,
        })

    if i % 200 == 0:
        torch.cuda.empty_cache()


Error in rouge_score: Hypothesis is empty.
Error in rouge_score: Hypothesis is empty.
Error in rouge_score: Hypothesis is empty.
Error in rouge_score: Hypothesis is empty.


In [34]:
for r in results[:5]:
    print("─── context ───")
    print(r["prefix"][-200:])       # show the tail of your context
    print("─── truth ───")
    print(r["truth"].rstrip())
    print("─ prediction ─")
    print(r["prediction"].rstrip())
    print()

─── context ───
ing[] args) {
        launch(args);
    }

    @Override
    public void start(Stage primaryStage) throws Exception {
        //TODO: the checks here take some time, so I would advise a splash screen

─── truth ───
        if (Database.isOK()) {
─ prediction ─
//TODO: or a progress bar

─── context ───
tors:
 *     IBM Corporation - initial API and implementation
 *     Lars Vogel <Lars.Vogel@vogella.com> - Bug 502576
 *******************************************************************************/

─── truth ───
package org.eclipse.swt.events;
─ prediction ─
package org.eclipse.rcp.intro.ui.wizards;

─── context ───
platforms. Traversal key
 * constants beginning with <code>TRAVERSE_</code> are defined
 * in the <code>SWT</code> class.
 * </p>
 *
 * @param e an event containing information about the traverse
 */

─── truth ───
void keyTraversed(TraverseEvent e);
─ prediction ─
void keyTraversed(TraverseEvent e);

─── context ───
hod and removed using
 * the <code>remov

In [35]:
import pandas as pd


In [36]:
metric_names = [
    "exact_match", "bleu", "levenshtein", "meteor",
    "rouge1_p","rouge1_r","rouge1_f",
    "rouge2_p","rouge2_r","rouge2_f",
    "rougeL_p","rougeL_r","rougeL_f",
]
metrics_rows = [row.tolist() for row in evaluation_results]

df_main    = pd.DataFrame(results)
df_metrics = pd.DataFrame(metrics_rows, columns=metric_names)

text_path   = "/content/drive/MyDrive/mellum_en_next_line_with_results.csv"
metric_path = "/content/drive/MyDrive/mellum_en_next_line_with_with_metrics.csv"

df_main.to_csv(text_path,   index=False, encoding="utf8")
df_metrics.to_csv(metric_path, index=False, encoding="utf8")

print(f"Saved {len(df_main)} text results")
print(f"Saved {len(df_metrics)} metrics results")

Saved 8457 text results
Saved 8457 metrics results
